# Experiments with Quantum QCNN

Construct and run a Quantum Convolutional Neural Network. The QCNN is constructed on the basis of number of features. SO if the number of features is x, we need x number of qbits in the circuit.

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import EstimatorQNN
from sklearn.model_selection import train_test_split
from qiskit_machine_learning.algorithms import QSVC

algorithm_globals.random_seed = 12345

/var/folders/4z/946_ygn50p3bc_rsjbq9cg240000gn/T/ipykernel_11598/2487926378.py:6: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B


In [2]:
# For dataset.
from sklearn.datasets import make_multilabel_classification, make_classification
from qiskit_algorithms.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import pandas as pd

In [4]:
# for data split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [5]:
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap

In [6]:
from qiskit.circuit.library import RealAmplitudes

In [7]:
results = {}

In [362]:
class quantumCNNMultiLabelAlgoTraining:
    """
    Class to train quantum algorithms for multiclass multilabel.
    """
    def __init__(self, no_of_features, no_of_samples, no_of_classes, no_of_labels, no_of_feature_map_reps, no_of_ansatz_reps, models = ['knn', 'qda', 'rf', 'lightgbm', 'svm']):
        self.no_of_features = no_of_features
        self.no_of_samples = no_of_samples
        self.no_of_labels = no_of_labels
        self.no_of_classes = no_of_classes
        self.models = models
        self.featuremap_reps = no_of_feature_map_reps
        self.ansatz_reps = no_of_ansatz_reps
        self.objective_func_vals = []
        print(f"the no of classes is {self.no_of_classes}")

    def data_generation(self):
        """
        Generate classification data using sklearn's data generation.\
        """
        n_samples=self.no_of_samples
        n_features=self.no_of_features
        n_classes=self.no_of_classes
        n_labels=self.no_of_labels
        print(n_samples)
        X, y = make_multilabel_classification(n_samples=n_samples, 
                                              n_features=n_features, 
                                              n_classes=n_classes, 
                                              n_labels=n_labels,
                                              random_state=algorithm_globals.random_seed
                                             )
        y_new = np.array([self.conv_to_int(val) for val in y])
        # y_new = y[:,0]*(2**4)+y[:,1]*(2**3)+y[:,2]*(2**2)+y[:,3]*(2**1)+y[:,4]*(2**0)
        # y_new = y[:,0]*(2**2)+y[:,1]*(2**1)+y[:,2]*(2**0)
        # y_new = y[:,0]*(2**1)+y[:,1]*(2**0)
        print(X.shape, y.shape, y_new.shape)
        print(X[:5], y[:5], y_new[:5])
        return X, y, y_new

    def int_to_hot(self, x):
        format = '{' + '0:0{:d}b'.format(self.no_of_classes) + '}'
        result = format.format(x)
        list_of_ints = [int(x) for x in result]
        return np.array(list_of_ints)

    def get_mapping(self):
        dictionary = {}
        numbers = list(range(0, 2**self.no_of_classes))
        for i in numbers:
            dictionary[i] = self.int_to_hot(i)
        return dictionary

    def conv_to_int(self, vector):
        val = "".join(vector.astype(str))
        value = int(val, 2)
        return value

    # We now define a two qubit unitary as defined in [3]
    def conv_circuit(self, params):
        target = QuantumCircuit(2)
        target.rz(-np.pi / 2, 1)
        target.cx(1, 0)
        target.rz(params[0], 0)
        target.ry(params[1], 1)
        target.cx(0, 1)
        target.ry(params[2], 1)
        target.cx(1, 0)
        target.rz(np.pi / 2, 0)
        return target

    def conv_layer(self, num_qubits, param_prefix):
        qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
        qubits = list(range(num_qubits))
        param_index = 0
        params = ParameterVector(param_prefix, length=num_qubits * 3)
        for q1, q2 in zip(qubits[0::2], qubits[1::2]):
            qc = qc.compose(self.conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
            qc.barrier()
            param_index += 3
        for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
            qc = qc.compose(self.conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
            qc.barrier()
            param_index += 3

        qc_inst = qc.to_instruction()

        qc = QuantumCircuit(num_qubits)
        qc.append(qc_inst, qubits)
        return qc

    # Defining pooling that will reduce the number of qbits(N)) at each application by half (N/2).
    def pool_circuit(self, params):
        target = QuantumCircuit(2)
        target.rz(-np.pi / 2, 1)
        target.cx(1, 0)
        target.rz(params[0], 0)
        target.ry(params[1], 1)
        target.cx(0, 1)
        target.ry(params[2], 1)

        return target

    def pool_layer(self, param_prefix):
        sources = self.split_integer_into_lists()[0]
        sinks = self.split_integer_into_lists()[1]
        num_qubits = len(sources) + len(sinks)
        qc = QuantumCircuit(num_qubits, name="Pooling Layer")
        param_index = 0
        params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
        for source, sink in zip(sources, sinks):
            qc = qc.compose(self.pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
            qc.barrier()
            param_index += 3

        qc_inst = qc.to_instruction()

        qc = QuantumCircuit(num_qubits)
        qc.append(qc_inst, range(num_qubits))
        return qc

    def split_integer_into_lists(self):
    # Calculate half of the integer
        half = self.no_of_features // 2
    
    # Create two lists based on half
        list1 = list(range(half))
        list2 = list(range(half, self.no_of_features))
    
        return list1, list2

    def get_mapping_old(self):
        mapping = {0: [0, 0, 0, 0, 0], 1: [0, 0, 0, 0, 1], 2: [0, 0, 0, 1, 0], 3: [0, 0, 0, 1, 1], 4: [0, 0, 1, 0, 0], 5: [0, 0, 1, 0, 1], 6: [0, 0, 1, 1, 0], 7: [0, 0, 1, 1, 1], 8: [0, 1, 0, 0, 0], 9: [0, 1, 0, 0, 1], 10: [0, 1, 0, 1, 0], 11: [0, 1, 0, 1, 1], 12: [0, 1, 1, 0, 0], 13: [0, 1, 1, 0, 1], 14: [0, 1, 1, 1, 0], 15: [0, 1, 1, 1, 1], 16: [1, 0, 0, 0, 0], 17: [1, 0, 0, 0, 1], 18: [1, 0, 0, 1, 0], 19: [1, 0, 0, 1, 1], 20: [1, 0, 1, 0, 0], 21: [1, 0, 1, 0, 1], 22: [1, 0, 1, 1, 0], 23: [1, 0, 1, 1, 1], 24: [1, 1, 0, 0, 0], 25: [1, 1, 0, 0, 1], 26: [1, 1, 0, 1, 0], 27: [1, 1, 0, 1, 1], 28: [1, 1, 1, 0, 0], 29: [1, 1, 1, 0, 1], 30: [1, 1, 1, 1, 0], 31: [1, 1, 1, 1, 1]}
        return mapping
        # mapping = {0: [0,0,0], 1: [0,0,1], 2:[0,1,0], 3:[0,1,1], 4:[1, 0, 0], 5:[1, 0, 1], 6:[1, 1, 0], 7:[1, 1, 1]}
        # mapping = {0: [0,0], 1: [0,1], 2: [1,0], 3:[1,1]}

    # def train_test_split(self):
    #     """
    #     splitting the data into train test split
    #     """
    #     ## Train test split
    #     # X, y = self.data_generation()
    #     train_features, test_features, train_labels, test_labels = train_test_split(
    #         X, y, train_size=0.8, random_state=algorithm_globals.random_seed
    #     )
    #     return train_features, test_features, train_labels, test_labels

    def generate_featuremap(self):
        """ Generating the feature map."""
        feature_map = ZZFeatureMap(feature_dimension=self.no_of_features, reps=self.featuremap_reps)
        return feature_map

    def generate_ansatz(self):
        """Generating ansatz."""
        ansatz = RealAmplitudes(num_qubits=self.no_of_features, reps=self.ansatz_reps)
        return ansatz

    def construct_qcnn_network(self):
        """Design a qcnn circuit based on the no of features."""
        feature_map = self.generate_featuremap() # feature map

        ansatz = self.generate_ansatz()
        counter = 1

        start_point = self.no_of_features
        while start_point % 2 == 0:
    
            # Convolutional Layer
            ansatz.compose(self.conv_layer(start_point, f"с{counter}"), list(range(start_point)), inplace=True)

            sources = split_integer_into_lists(start_point)[0]
            sinks = split_integer_into_lists(start_point)[1]
            ansatz.compose(self.pool_layer(f"p{counter}"), list(range(start_point)), inplace=True)

            counter += 1
            start_point = start_point // 2

        # Combining the feature map and ansatz
        circuit = QuantumCircuit(self.no_of_features)
        circuit.compose(feature_map, range(self.no_of_features), inplace=True)
        circuit.compose(ansatz, range(self.no_of_features), inplace=True)
        return circuit, feature_map.parameters, ansatz.parameters

    def get_observable(self):
        observable = SparsePauliOp.from_list([("Z" + "I" * (self.no_of_features-1), 2)])
        return observable
        
    def callback_graph(self, weights, obj_func_eval):
        clear_output(wait=True)
        self.objective_func_vals.append(obj_func_eval)
        plt.title("Objective function value against iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Objective function value")
        plt.plot(range(len(self.objective_func_vals)), self.objective_func_vals)
        plt.show()

    def quantum_training(self):
        X, y, y_new = self.data_generation()
        print(X.shape, y.shape, y_new.shape)
        # best_model = compare_models(include=self.models, sort='f1')
        # print(best_model)
        mapping = self.get_mapping()
        optimizer = COBYLA(maxiter=100)
        model = 'QCNN'
        QCNN_network = self.construct_qcnn_network()
        circuit = QCNN_network[0]
        input_parameters = QCNN_network[1]
        weight_parameters = QCNN_network[2]
        qnn = EstimatorQNN(
            circuit=circuit.decompose(),
            observables=self.get_observable(),
            input_params=input_parameters,
            weight_params=weight_parameters
        )
        classifier = NeuralNetworkClassifier(
        qnn,
        optimizer=COBYLA(maxiter=400),  # Set max iterations here
        callback=self.callback_graph,
        one_hot=True
        # initial_point=initial_point,
        )

        regressor = NeuralNetworkRegressor(
        neural_network=qnn,
        loss="squared_error",
        optimizer=L_BFGS_B(maxiter=30),
        callback=self.callback_graph,
        )
        
        # clear objective value history
        objective_func_vals = []
        
        start = time.time()
        classifier.fit(X,np.array([1]))
        elapsed = time.time() - start
        
        print(f"Training time: {round(elapsed)} seconds")
        # testing the QSVC scores
        predictions = classifier.predict(X)

        # Use list comprehension to create the new array
        # print(f"The type of predictions is: {type(predictions)}")
        # predictions_to_list = predictions.tolist()
        # predictions_labels_final = np.array([mapping[val] for val in predictions_to_list])
        # print(y.shape, predictions_labels_final.shape)
        # print(predictions_labels_final[:5])
        accuracy_scores = accuracy_score(y, predictions)
        f1_scores = f1_score(y, predictions, average='weighted')
        results.setdefault('Model', []).append(model)
        results.setdefault('No of features', []).append(self.no_of_features)
        results.setdefault('No of samples', []).append(self.no_of_samples)
        results.setdefault('No of classes', []).append(self.no_of_classes)
        results.setdefault('No of labels', []).append(self.no_of_labels)
        results.setdefault('No of feature map reps', []).append(self.featuremap_reps)
        results.setdefault('No of ansatz reps', []).append(self.ansatz_reps)
        results.setdefault('Accuracy', []).append(accuracy_scores)
        results.setdefault('F1', []).append(f1_scores)
        # df = pd.DataFrame(results)
        # df.to_csv(f'''results/Quantum-MC-ML-features_{self.no_of_features}-samples_{self.no_of_samples}-classes_{self.no_of_classes}-labels_{self.no_of_labels}-featuremapReps_{self.featuremap_reps}-ansatzReps_{self.ansatz_reps}.csv''', index=False)
        # return df

In [363]:
# # Defining pooling that will reduce the number of qbits(N)) at each application by half (N/2).
# def pool_circuit(params):
#     target = QuantumCircuit(8)
#     target.rz(-np.pi / 2, 1)
#     target.cx(1, 0)
#     target.rz(params[0], 0)
#     target.ry(params[1], 1)
#     target.cx(0, 1)
#     target.ry(params[2], 1)

#     return target

In [364]:
# def pool_layer(sources, sinks, param_prefix):
#     num_qubits = len(sources) + len(sinks)
#     qc = QuantumCircuit(num_qubits, name="Pooling Layer")
#     param_index = 0
#     params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
#     for source, sink in zip(sources, sinks):
#         qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
#         qc.barrier()
#         param_index += 3

#     qc_inst = qc.to_instruction()

#     qc = QuantumCircuit(num_qubits)
#     qc.append(qc_inst, range(num_qubits))
#     return qc

In [365]:
# sources = split_integer_into_lists(8)[0]
# sinks = split_integer_into_lists(8)[1]
# circuit = pool_layer(sources, sinks, "θ")
# circuit.decompose().draw("mpl")

In [11]:
no_of_features = [2]
no_of_samples = [1024]
no_of_classes = [3]
no_of_labels = [3]
no_of_feature_map_reps = [3]
no_of_ansatz_reps = [3]

In [12]:
for lab in no_of_labels:
    for clas in no_of_classes:
        for feature in no_of_features:
            for sample in no_of_samples:
                for featuremap_reps in no_of_feature_map_reps:
                    for ansatz_reps in no_of_ansatz_reps:
                        training_object = quantumCNNMultiLabelAlgoTraining(feature, sample, clas, lab, featuremap_reps, ansatz_reps)
                        training_object.quantum_training()

NameError: name 'quantumCNNMultiLabelAlgoTraining' is not defined

In [361]:
len(np.array([1]).shape)

1


-----

## Trying different approach.

In [8]:
def split_integer_into_lists(n):
    # Calculate half of the integer
    half = n // 2
    
    # Create two lists based on half
    list1 = list(range(half))
    list2 = list(range(half, n))
    
    return list1, list2

In [9]:
def conv_to_int(vector):
        val = "".join(vector.astype(str))
        value = int(val, 2)
        return value

In [10]:
def int_to_hot(x, no_of_classes):
        format = '{' + '0:0{:d}b'.format(no_of_classes) + '}'
        result = format.format(x)
        list_of_ints = [int(x) for x in result]
        return np.array(list_of_ints)

In [11]:
def get_mapping(no_of_classes):
        dictionary = {}
        numbers = list(range(0, 2**no_of_classes))
        for i in numbers:
            dictionary[i] = int_to_hot(i, no_of_classes)
        return dictionary

In [12]:
# get data
def data_generation(n_samples, n_features, n_classes, n_labels):
        """
        Generate classification data using sklearn's data generation.\
        """
        X, y = make_multilabel_classification(n_samples=n_samples, 
                                              n_features=n_features, 
                                              n_classes=n_classes, 
                                              n_labels=n_labels,
                                              random_state=algorithm_globals.random_seed
                                             )
        y_new = np.array([conv_to_int(val) for val in y])
        # y_new = y[:,0]*(2**4)+y[:,1]*(2**3)+y[:,2]*(2**2)+y[:,3]*(2**1)+y[:,4]*(2**0)
        # y_new = y[:,0]*(2**2)+y[:,1]*(2**1)+y[:,2]*(2**0)
        # y_new = y[:,0]*(2**1)+y[:,1]*(2**0)
        print(X.shape, y.shape, y_new.shape)
        print(X[:5], y[:5], y_new[:5])
        return X, y, y_new

In [13]:
# We now define a two qubit unitary as defined in [3]
def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target

In [14]:
def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits * 3)
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, qubits)
    return qc

In [15]:
def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)

    return target

In [16]:
def pool_layer(sources, sinks, param_prefix):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, range(num_qubits))
    return qc

In [17]:
def construct_qcnn_network(no_feature):
    """Design a qcnn circuit based on the no of features."""
    feature_map = ZZFeatureMap(no_feature)

    ansatz = RealAmplitudes(no_feature)
    counter = 1

    start_point = no_feature
    sources, sinks = split_integer_into_lists(start_point)
    while start_point % 2 == 0:
    
        # Convolutional Layer
        ansatz.compose(conv_layer(start_point, f"с{counter}"), list(range(start_point)), inplace=True)

        sources = split_integer_into_lists(start_point)[0]
        sinks = split_integer_into_lists(start_point)[1]
        ansatz.compose(pool_layer(sources, sinks, f"p{counter}"), list(range(start_point)), inplace=True)

        counter += 1
        start_point = start_point // 2

    # Combining the feature map and ansatz
    circuit = QuantumCircuit(no_feature)
    circuit.compose(feature_map, range(no_feature), inplace=True)
    circuit.compose(ansatz, range(no_feature), inplace=True)
    return circuit, feature_map.parameters, ansatz.parameters

In [18]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [19]:
results = {}
model = 'Quantum CNN'
no_of_samples = 1024
no_of_features = [8] # as much as quantum side allows
no_of_classes = [1] # look for quantum side
no_of_labels = [1] # starting from 2, till max no of classes.
no_of_featuremap_reps = [2]

In [20]:
def generate_dataset(num_images):
    images = []
    labels = []
    hor_array = np.zeros((6, 8))
    ver_array = np.zeros((4, 8))

    j = 0
    for i in range(0, 7):
        if i != 3:
            hor_array[j][i] = np.pi / 2
            hor_array[j][i + 1] = np.pi / 2
            j += 1

    j = 0
    for i in range(0, 4):
        ver_array[j][i] = np.pi / 2
        ver_array[j][i + 4] = np.pi / 2
        j += 1

    for n in range(num_images):
        rng = algorithm_globals.random.integers(0, 2)
        if rng == 0:
            labels.append(-1)
            random_image = algorithm_globals.random.integers(0, 6)
            images.append(np.array(hor_array[random_image]))
        elif rng == 1:
            labels.append(1)
            random_image = algorithm_globals.random.integers(0, 4)
            images.append(np.array(ver_array[random_image]))

        # Create noise
        for i in range(8):
            if images[-1][i] == 0:
                images[-1][i] = algorithm_globals.random.uniform(0, np.pi / 4)
    return images, labels

In [21]:
images, labels = generate_dataset(50)

/var/folders/4z/946_ygn50p3bc_rsjbq9cg240000gn/T/ipykernel_11598/593309357.py:21: DeprecationWarning: Using random that is seeded via qiskit.utils algorithm_globals is deprecated since version 0.2.0. Instead set random_seed directly to qiskit_algorithms.utils algorithm_globals.
  rng = algorithm_globals.random.integers(0, 2)
/var/folders/4z/946_ygn50p3bc_rsjbq9cg240000gn/T/ipykernel_11598/593309357.py:28: DeprecationWarning: Using random that is seeded via qiskit.utils algorithm_globals is deprecated since version 0.2.0. Instead set random_seed directly to qiskit_algorithms.utils algorithm_globals.
  random_image = algorithm_globals.random.integers(0, 4)
/var/folders/4z/946_ygn50p3bc_rsjbq9cg240000gn/T/ipykernel_11598/593309357.py:34: DeprecationWarning: Using random that is seeded via qiskit.utils algorithm_globals is deprecated since version 0.2.0. Instead set random_seed directly to qiskit_algorithms.utils algorithm_globals.
  images[-1][i] = algorithm_globals.random.uniform(0, np.p

In [22]:
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.3
)

In [23]:
x = np.asarray(train_images)
y = np.asarray(train_labels)

In [24]:
x.shape, y.shape

((35, 8), (35,))

In [15]:
for no_feature in no_of_features:
    for reps in no_of_featuremap_reps:
        for no_class in no_of_classes:
            for no_label in no_of_labels:
                if no_label <= no_class:
                    X, y, y_new = data_generation(no_of_samples, no_feature, no_class, no_label)
                    print(X.shape, y.shape, y_new.shape)
                    QCNN_network = construct_qcnn_network(no_feature)
                    circuit = QCNN_network[0]
                    input_params = QCNN_network[1]
                    weight_params = QCNN_network[2]
                    observable = SparsePauliOp.from_list([("Z" + "I" * 7, 1)])
                    qnn = EstimatorQNN(circuit=circuit.decompose(), observables=observable, 
                                       input_params=input_params, weight_params=weight_params)
                    classifier = NeuralNetworkClassifier(qnn, 
                                                         optimizer=COBYLA(maxiter=200),
                                                         callback=callback_graph,
                                                         one_hot=True
                                                        )
                    classifier.fit(X, y)
                    

NameError: name 'make_multilabel_classification' is not defined

In [ ]:
for no_feature in no_of_features:
    for reps in no_of_featuremap_reps:
        for no_class in no_of_classes:
            for no_label in no_of_labels:
                if no_label <= no_class:
                    # X, y, y_new = data_generation(no_of_samples, no_feature, no_class, no_label)
                    # print(X.shape, y.shape, y_new.shape)
                    feature_map = ZFeatureMap(8)

                    ansatz = QuantumCircuit(8, name="Ansatz")

                    # First Convolutional Layer
                    ansatz.compose(conv_layer(8, "с1"), list(range(8)), inplace=True)

                    # First Pooling Layer
                    ansatz.compose(pool_layer([0, 1, 2, 3], [4, 5, 6, 7], "p1"), list(range(8)), inplace=True)

                    # Second Convolutional Layer
                    ansatz.compose(conv_layer(4, "c2"), list(range(4, 8)), inplace=True)

                    # Second Pooling Layer
                    ansatz.compose(pool_layer([0, 1], [2, 3], "p2"), list(range(4, 8)), inplace=True)

                    # Third Convolutional Layer
                    ansatz.compose(conv_layer(2, "c3"), list(range(6, 8)), inplace=True)

                    # Third Pooling Layer
                    ansatz.compose(pool_layer([0], [1], "p3"), list(range(6, 8)), inplace=True)

                    # Combining the feature map and ansatz
                    circuit = QuantumCircuit(8)
                    circuit.compose(feature_map, range(8), inplace=True)
                    circuit.compose(ansatz, range(8), inplace=True)

                    observable = SparsePauliOp.from_list([("Z" + "I" * 7, 1)])
                    # qnn = EstimatorQNN(circuit=circuit.decompose(),
                    #                    input_params=feature_map.parameters,
                    #                    weight_params=ansatz.parameters,)
                    qnn = EstimatorQNN(circuit=circuit.decompose(), observables=observable)
                    classifier = NeuralNetworkClassifier(qnn, 
                                                         optimizer=COBYLA(maxiter=200),
                                                         callback=callback_graph,)
                    classifier.fit(x, y)

Traceback (most recent call last):
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
  File "/Users/prasham/miniconda3/envs/quantum-base/lib/python3.9/site-packages/scipy/optimize/_cobyla_py.py", line 258, in calcfc
    f = fun(np.copy(x), *args)
  File "/Users/prasham/miniconda3/envs/quantum-base/lib/python3.9/site-packages/qiskit_machine_learning/algorithms/trainable_model.py", line 271, in objective
    objective_value = function.objective(objective_weights)
  File "/Users/prasham/miniconda3/envs/quantum-base/lib/python3.9/site-packages/qiskit_machine_learning/algorithms/objective_functions.py", line 115, in objective
    predict = self._neural_network_forward(weights)
  File "/Users/prasham/miniconda3/envs/quantum-base/lib/python3.9/site-packages/qiskit_machine_learning/algorithms/objective_functions.py", line 102, in